In [ ]:
import numpy as np

In [ ]:
!pip install -q sentence-transformers

In [ ]:
!pip install -q transformers datasets

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader
import torch

In [ ]:
train_path = '/content/drive/MyDrive/quora dataset/train.csv'

test_path = '/content/drive/MyDrive/quora dataset/test.csv'

In [ ]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

<ipython-input-22-47f939b655d8>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(test_path)


In [ ]:
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
train_data = train_data.sample(frac=0.1, random_state=42)
test_data = test_data.sample(frac=0.1, random_state=42)

In [ ]:
import re

def preprocess(text):
    # Check if the input is a string; if not, return an empty string
    if isinstance(text, str):
        text = text.lower()  # Lowercase
        text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters and numbers
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace

        # Handle contractions
        contractions = {
            "can't": "cannot", "won't": "will not", "n't": " not", "'re": " are", "'s": " is",
            "'d": " would", "'ll": " will", "'t": " not", "'ve": " have", "'m": " am"
        }
        for contraction, full_form in contractions.items():
            text = text.replace(contraction, full_form)
        return text
    else:
        return ""  # Return empty string if the input is not a string

# Apply preprocessing to the train and test data
train_data['question1'] = train_data['question1'].apply(preprocess)
train_data['question2'] = train_data['question2'].apply(preprocess)

test_data['question1'] = test_data['question1'].apply(preprocess)
test_data['question2'] = test_data['question2'].apply(preprocess)


In [ ]:
test_data.dtypes

,0
test_id,object
question1,object
question2,object


In [ ]:
train_examples = [InputExample(texts=[row['question1'], row['question2']], label=float(row['is_duplicate']))
                  for _, row in train_data.iterrows()]

In [ ]:
test_examples = [InputExample(texts=[row['question1'], row['question2']])
                 for _, row in test_data.iterrows()]

In [ ]:
from sklearn.metrics import f1_score

bi-encoder model

In [ ]:
def evaluate_bi_encoder(model, test_examples):
    predictions = []
    for example in test_examples:
        embeddings1 = model.encode(example.texts[0], convert_to_tensor=True)
        embeddings2 = model.encode(example.texts[1], convert_to_tensor=True)

        # Ensure correct dimensions for cosine similarity
        embeddings1 = embeddings1.unsqueeze(0) if len(embeddings1.shape) == 1 else embeddings1
        embeddings2 = embeddings2.unsqueeze(0) if len(embeddings2.shape) == 1 else embeddings2

        cosine_sim = torch.nn.functional.cosine_similarity(embeddings1, embeddings2).item()
        predictions.append(1 if cosine_sim > 0.5 else 0)

    true_labels = [example.label for example in test_examples]
    return f1_score(true_labels, predictions)


Train a bi-encoder using Cosine Similarity Loss

In [ ]:
model_cosine = SentenceTransformer('paraphrase-MiniLM-L6-v2')
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
loss_cosine = losses.CosineSimilarityLoss(model=model_cosine)
model_cosine.fit(train_objectives=[(train_dataloader, loss_cosine)], epochs=3, warmup_steps=100)
f1_cosine = evaluate_bi_encoder(model_cosine, train_examples)
print(f"F1-Score (Cosine Similarity): {f1_cosine}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Step,Training Loss
500,0.172000
1000,0.151000
1500,0.147700
2000,0.141400
2500,0.143000
3000,0.125500
3500,0.120300
4000,0.121900
4500,0.119800
5000,0.120200


F1-Score (Cosine Similarity): 0.8527485731450886


Train a bi-encoder using Contrastive Loss

In [ ]:
model_contrastive = SentenceTransformer('paraphrase-MiniLM-L6-v2')
loss_contrastive = losses.ContrastiveLoss(model=model_contrastive)
model_contrastive.fit(train_objectives=[(train_dataloader, loss_contrastive)], epochs=3, warmup_steps=100)
f1_contrastive = evaluate_bi_encoder(model_contrastive, train_examples)
print(f"F1-Score (Contrastive Loss): {f1_contrastive}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Step,Training Loss
500,0.017100
1000,0.016000
1500,0.015600
2000,0.014900
2500,0.015000
3000,0.012800
3500,0.012200
4000,0.012500
4500,0.012200
5000,0.012200


F1-Score (Contrastive Loss): 0.714295964983616


Train a bi-encoder using Multiple Negative Ranking Loss

In [ ]:
model_mnr = SentenceTransformer('paraphrase-MiniLM-L6-v2')
loss_mnr = losses.MultipleNegativesRankingLoss(model=model_mnr)
model_mnr.fit(train_objectives=[(train_dataloader, loss_mnr)], epochs=3, warmup_steps=100)
f1_mnr = evaluate_bi_encoder(model_mnr, train_examples)
print(f"F1-Score (Multiple Negative Ranking Loss): {f1_mnr}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Step,Training Loss
500,0.373700
1000,0.337600
1500,0.330000
2000,0.343100
2500,0.299600
3000,0.247900
3500,0.214500
4000,0.223300
4500,0.236300
5000,0.216600


F1-Score (Multiple Negative Ranking Loss): 0.6228395449688894


Train a cross-encoder

In [ ]:
from sentence_transformers import CrossEncoder

In [ ]:
# Convert training data into InputExample format
train_examples = [InputExample(texts=[row['question1'], row['question2']], label=float(row['is_duplicate']))
                  for _, row in train_data.iterrows()]

# Convert test data (no labels) for inference
test_examples = [[row['question1'], row['question2']] for _, row in test_data.iterrows()]



In [ ]:
train_dataloader = DataLoader(train_examples, batch_size=16, shuffle=True)

In [ ]:
# Load the pre-trained cross-encoder model
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

In [ ]:
# Train the cross-encoder using the DataLoader
cross_encoder.fit(train_dataloader, epochs=3)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2527 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2527 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2527 [00:00<?, ?it/s]

In [ ]:
# Function to evaluate cross-encoder
def evaluate_cross_encoder(model, test_data, true_labels):
    predictions = model.predict(test_data)
    predicted_labels = [1 if pred > 0.5 else 0 for pred in predictions]
    return f1_score(true_labels, predicted_labels)

In [ ]:
test_labels = list(train_data['is_duplicate'][:len(test_examples)])

In [ ]:
# Limit test labels to match the number of test examples
test_labels = train_labels_cross[:len(test_examples)]  # Use a portion of the training labels for testing

# Evaluate on the matching set
f1_score_result = evaluate_cross_encoder(cross_encoder, test_examples, test_labels)
print(f"F1-Score (Cross-Encoder): {f1_score_result}")


ValueError: Found input variables with inconsistent numbers of samples: [40429, 356348]